In [1]:
# LOW_PRB

"""
PBR : (시가총액) / (총 자본) = (주가 * 총 주식수) / (총 자산 - 총 부채)
"""

'\nPBR : (시가총액) / (총 자본) = (주가 * 총 주식수) / (총 자산 - 총 부채)\n'

In [2]:
# imports

import kquant as kq
import datetime as dt


# TMP

import random

In [3]:
# API_account
def set_api_account():
    kq.set_api("KRX2308020", "EQDkUcyI3dK6oIAXqAR8BXOK4bKxHHmH")
    return None

In [4]:
set_api_account()

In [5]:
# LOAD_DATA
# class DATA_LOADER:
class SYMBOL_LOADER:
    @staticmethod
    def load_symbols_df():
        symbols_df = kq.symbol_stock()
        return symbols_df

    class SYMBOL_FILTER:
        @staticmethod
        def filter__market(symbols_df):
            filtered_symbols_df = symbols_df[
                (symbols_df["MARKET"].isin(["코스닥", "유가증권"]))
            ].copy()
            return filtered_symbols_df

        @staticmethod
        def filter__admin_issue(symbols_df):
            filtered_symbols_df = symbols_df[(symbols_df["ADMIN_ISSUE"] == 0)].copy()
            return filtered_symbols_df

        @staticmethod
        def filter_sec_type(symbols_df):
            filtered_symbols_df = symbols_df[
                symbols_df["SEC_TYPE"].isin(["ST", "EF", "EN"])
            ].copy()
            return filtered_symbols_df

    def filter_symbols_df(self, symbols_df):
        symbol_filter = self.SYMBOL_FILTER()
        filtered_symbols_df = symbol_filter.filter__market(symbols_df)
        filtered_symbols_df = symbol_filter.filter__admin_issue(filtered_symbols_df)
        filtered_symbols_df = symbol_filter.filter_sec_type(filtered_symbols_df)
        return filtered_symbols_df

    @staticmethod
    def get_symbols(symbols_df):
        symbols = sorted(set(symbols_df["SYMBOL"]))
        return symbols

    # SYMBOL_LOADER PIPELINE
    def __call__(self):
        symbols_df = self.load_symbols_df()
        filtered_symbols_df = self.filter_symbols_df(symbols_df)
        symbols = self.get_symbols(filtered_symbols_df)
        return symbols

In [6]:
class FUNDAMENTAL_LOADER:
    def __init__(self, symbol, date) -> None:
        self.symbol = symbol
        self.date = date
        self.daily_stock_df = kq.daily_stock(
            symbol,
            start_date=date - dt.timedelta(days=7),
            end_date=date,
        )

    def load_recent_close(self):
        daily_stock_df = self.daily_stock_df
        _close = daily_stock_df.sort_values("DATE").tail(1)["CLOSE"].values[0]
        return _close

    def load_recent_marketcap(self):
        daily_stock_df = self.daily_stock_df
        _marketcap = daily_stock_df.sort_values("DATE").tail(1)["MARKETCAP"].values[0]
        return _marketcap

    def load_recent_netprofit(self):
        netprofit_df = kq.account_history(self.symbol, "122700")
        netprofit_df.sort_values("YEARMONTH", inplace=True)
        _netprofit = netprofit_df.tail(1)["VALUE"].values[0]
        return _netprofit

    def load_recent_capital(self):
        capital_df = kq.account_history(self.symbol, "115000")
        capital_df.sort_values("YEARMONTH", inplace=True)
        _capital = capital_df.tail(1)["VALUE"].values[0]
        return _capital

    def __call__(self):
        _close = self.load_recent_close()
        _marketcap = self.load_recent_marketcap()
        _netprofit = self.load_recent_netprofit()
        _capital = self.load_recent_capital()
        return {
            "symbol": self.symbol,
            "close": _close,
            "marketcap": _marketcap,
            "netprofit": _netprofit,
            "capital": _capital,
        }

In [7]:
class STATUS_LOADER:
    def __init__(self, dict_df_result, dict_df_position) -> None:
        self.dict_df_result = dict_df_result
        self.dict_df_position = dict_df_position

    def get_current_cash(self):
        _dict_df_result = self.dict_df_result
        try:
            _df_result_total = _dict_df_result["TOTAL"]
            _current_cash = (
                _df_result_total.sort_values("DATE").tail(1)["CASH"].values[0]
            )
            return _current_cash
        except:
            return 1_000_000_000

    def get_current_symbol_df(self):
        current_symbol_list = list()
        _dict_df_result = self.dict_df_result
        _dict_df_position = self.dict_df_position

        _total_symbols = sorted(_dict_df_position.keys())

        for _symbol in _total_symbols:
            try:
                _symbol_result_df = _dict_df_result[_symbol]
                _symbol_position_df = _dict_df_position[_symbol]

                _current_price = (
                    _symbol_result_df.sort_values("DATE").tail(1)["PRICE"].values[0]
                )
                _trade_price = _symbol_position_df["TRADE_PRICE"].values[0]
                _current_qty = _symbol_position_df["QTY"].values[0]

                current_symbol_list.append(
                    {
                        "symbol": _symbol,
                        "current_qty": _current_qty,
                        "current_price": _current_price,
                        "trade_price": _trade_price,
                    }
                )
            except:
                pass
        return pd.DataFrame(current_symbol_list)

In [8]:
# Signiture
import pandas as pd
import datetime as dt
import logging


def trade_func(
    date: dt.date,
    dict_df_result: dict[str, pd.DataFrame],
    dict_df_position: dict[str, pd.DataFrame],
    logger: logging.Logger,
) -> list[tuple[str, int]]:
    daily_invest_money = 10_000_000
    """LOAD DATA"""

    """ LOAD DATA (SYMBOL) """
    symbol_loader = SYMBOL_LOADER()
    total_symbols = symbol_loader()

    # tmp
    total_symbols = random.sample(total_symbols, 100)

    """ LOAD DATA (FUNDAMENTAL) """
    fundamental_data_results = list()

    for symbol in total_symbols:
        try:
            fundamental_loader = FUNDAMENTAL_LOADER(symbol=symbol, date=date)
            _fundamental_data = fundamental_loader()
            fundamental_data_results.append(_fundamental_data)
        except:
            pass

    fundamental_df = pd.DataFrame(fundamental_data_results)
    fundamental_df["pbr"] = fundamental_df["marketcap"] / fundamental_df["capital"]
    low_pbr_df = fundamental_df.nsmallest(5, "pbr")
    low_pbr_df["pbr_weight"] = low_pbr_df["pbr"].sum() / low_pbr_df["pbr"]
    low_pbr_df["price_invest"] = (
        low_pbr_df["pbr_weight"] / low_pbr_df["pbr_weight"].sum()
    ) * daily_invest_money
    low_pbr_df["cnt_invest"] = low_pbr_df["price_invest"] // low_pbr_df["close"]
    symbols_and_orders = list(
        low_pbr_df.set_index("symbol")["cnt_invest"].astype(int).to_dict().items()
    )
    print(date)
    return symbols_and_orders

In [9]:
dict_df_result = kq.backtest_stock_port_daily(
    trade_func,
    "2023-08-11",  # 실제 심사에서는 투자기간 시작일
    "2023-09-13",  # 실제 심사에서는 투자기간 종료일
    init_cash=1_000_000_000,  # 10억원
)

[2023-08-11] 종목: 000140, 주문전 보유수량:      0 주문수량:    274, 매매수량:    274, 주문후 보유수량:    274
[2023-08-11] 종목: 001940, 주문전 보유수량:      0 주문수량:    125, 매매수량:    125, 주문후 보유수량:    125
[2023-08-11] 종목: 004020, 주문전 보유수량:      0 주문수량:     51, 매매수량:     51, 주문후 보유수량:     51
[2023-08-11] 종목: 001130, 주문전 보유수량:      0 주문수량:     13, 매매수량:     13, 주문후 보유수량:     13


2023-08-11


[2023-08-11] 종목: 093050, 주문전 보유수량:      0 주문수량:    117, 매매수량:    117, 주문후 보유수량:    117
[2023-08-14] 종목: 023590, 주문전 보유수량:      0 주문수량:    162, 매매수량:    162, 주문후 보유수량:    162
[2023-08-14] 종목: 267250, 주문전 보유수량:      0 주문수량:     35, 매매수량:     35, 주문후 보유수량:     35
[2023-08-14] 종목: 078860, 주문전 보유수량:      0 주문수량:  7,205, 매매수량:  7,205, 주문후 보유수량:  7,205
[2023-08-14] 종목: 069960, 주문전 보유수량:      0 주문수량:     24, 매매수량:     24, 주문후 보유수량:     24


2023-08-14


[2023-08-14] 종목: 031310, 주문전 보유수량:      0 주문수량:    520, 매매수량:    520, 주문후 보유수량:    520
[2023-08-16] <KQuantNotAllowShort> 공매도 금지 오류: 매도 수량이 현재 보유수량보다 큼
[2023-08-16] 종목: 056730, 주문전 보유수량:      0 주문수량:    -15, 매매수량:      0, 주문후 보유수량:      0
[2023-08-16] 종목: 000880, 주문전 보유수량:      0 주문수량:    154, 매매수량:    154, 주문후 보유수량:    154
[2023-08-16] 종목: 092230, 주문전 보유수량:      0 주문수량:     49, 매매수량:     49, 주문후 보유수량:     49
[2023-08-16] 종목: 004150, 주문전 보유수량:      0 주문수량:    682, 매매수량:    682, 주문후 보유수량:    682


...

[2023-09-11] 종목: 009200, 주문전 보유수량:      0 주문수량:  1,134, 매매수량:  1,134, 주문후 보유수량:  1,134
[2023-09-11] 종목: 004150, 주문전 보유수량:    682 주문수량:    649, 매매수량:    649, 주문후 보유수량:  1,331
[2023-09-11] 종목: 004360, 주문전 보유수량:      0 주문수량:    132, 매매수량:    132, 주문후 보유수량:    132
[2023-09-11] 종목: 000540, 주문전 보유수량:      0 주문수량:    395, 매매수량:    395, 주문후 보유수량:    395
[2023-09-12] 종목: 036530, 주문전 보유수량:      0 주문수량:    193, 매매수량:    193, 주문후 보유수량:    193
[2023-09-12] 종목: 002320, 주문전 보유수량:    103 주문수량:     96, 매매수량:     96, 주문후 보유수량:    199
[2023-09-12] 종목: 363280, 주문전 보유수량:    350 주문수량:    280, 매매수량:    280, 주문후 보유수량:    630
[2023-09-12] 종목: 000150, 주문전 보유수량:      0 주문수량:     13, 매매수량:     13, 주문후 보유수량:     13


2023-09-12


[2023-09-12] 종목: 035760, 주문전 보유수량:      0 주문수량:     21, 매매수량:     21, 주문후 보유수량:     21
[2023-09-13] 종목: 003380, 주문전 보유수량:      0 주문수량:    371, 매매수량:    371, 주문후 보유수량:    371
[2023-09-13] 종목: 023530, 주문전 보유수량:     27 주문수량:     29, 매매수량:     29, 주문후 보유수량:     56
[2023-09-13] 종목: 016450, 주문전 보유수량:    368 주문수량:    418, 매매수량:    418, 주문후 보유수량:    786
[2023-09-13] 종목: 003200, 주문전 보유수량:      0 주문수량:    190, 매매수량:    190, 주문후 보유수량:    190


2023-09-13


[2023-09-13] 종목: 091340, 주문전 보유수량:      0 주문수량:    566, 매매수량:    566, 주문후 보유수량:    566


In [10]:
dict_df_result['TOTAL']

,DATE,SYMBOL,PRICE,ORDER,QTY,TRADE_PRICE,POSITION,AVG_PRICE,FEE,TRADE_TAX,SLIPPAGE,CASHFLOW,CASH,HIST_VALUE,STOCK_VALUE,TOTAL_VALUE,REAL_PROFIT,UNREAL_PROFIT,PROFIT,HIGHWATERMARK,DRAWDOWN
0,2023-08-11,TOTAL,0,0,0,0,580,0.0000,0,0,0,"-9,875,370","990,124,630","9,875,370","9,875,370","1,000,000,000",0,0,0,"1,000,000,000",0
1,2023-08-14,TOTAL,0,0,0,0,"8,526",0.0000,0,0,0,"-9,959,310","980,165,320","19,834,680","19,844,190","1,000,009,510",0,"9,510","9,510","1,000,009,510",0
2,2023-08-16,TOTAL,0,0,0,0,"15,719",0.0000,0,0,0,"-9,958,302","970,207,018","29,792,982","29,451,562","999,658,580",0,"-341,420","-341,420","1,000,009,510","350,930"
3,2023-08-17,TOTAL,0,0,0,0,"17,073",0.0000,0,0,0,"-9,891,870","960,315,148","39,684,852","39,064,834","999,379,982",0,"-620,018","-620,018","1,000,009,510","629,528"
4,2023-08-18,TOTAL,0,0,0,0,"18,834",0.0000,0,0,0,"-9,990,530","950,324,618","49,675,382","48,903,287","999,227,905",0,"-772,095","-772,095","1,000,009,510","781,605"
5,2023-08-21,TOTAL,0,0,0,0,"22,290",0.0000,0,0,0,"-9,950,070","940,374,548","59,625,452","58,928,978","999,303,526",0,"-696,474","-696,474","1,000,009,510","705,984"
6,2023-08-22,TOTAL,0,0,0,0,"30,356",0.0000,0,0,0,"-9,992,254","930,382,294","69,617,706","68,864,706","999,247,000",0,"-753,000","-753,000","1,000,009,510","762,510"
7,2023-08-23,TOTAL,0,0,0,0,"31,795",0.0000,0,0,0,"-9,910,140","920,472,154","79,527,846","78,599,900","999,072,054",0,"-927,946","-927,946","1,000,009,510","937,456"
8,2023-08-24,TOTAL,0,0,0,0,"33,246",0.0000,0,0,0,"-9,981,820","910,490,334","89,509,666","88,804,219","999,294,553",0,"-705,447","-705,447","1,000,009,510","714,957"
9,2023-08-25,TOTAL,0,0,0,0,"34,591",0.0000,0,0,0,"-9,973,880","900,516,454","99,483,546","98,791,293","999,307,747",0,"-692,253","-692,253","1,000,009,510","701,763"
